In [56]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

In [29]:
# generate random data
def generate_data(xmin, xmax, Delta, noise):
    # calculate f=sinx1 + cosx2
    x1 = np.arange(xmin, xmax+Delta, Delta)
    x2 = np.arange(xmin, xmax+Delta, Delta)
    
    x1, x2 = np.meshgrid(x1, x2)
    f = np.sin(x1) + np.cos(x2)
   
    # add random noise
    np.random.seed(200) # for reproducibility
    for i in range(len(f)):
        for j in range(len(f[0])):
            f[i][j] = f[i][j] + np.random.uniform(-noise, noise) # random noise addition
    return x1, x2, f

x1,x2,f = generate_data(-10, 10, 1.0, 0.5)

In [41]:
# convert data to (X,y)
def prepare_data_to_KRR(x1, x2, f):
    X = []
    for i in range(len(f)):
        for j in range(len(f)):
            X_term = []
            X_term.append(x1[i][j])
            X_term.append(x2[i][j])
            X.append(X_term)
            
    y = f.flatten()
    X = np.array(X)
    y = np.array(y)
    return X,y

X,y = prepare_data_to_KRR(x1,x2,f)

In [66]:
#kernel ridge regression
def KRR_function(hyperparams, X, y):
    # assign hyperparameters
    alpha_value,gamma_value = hyperparams
    
    y_pred_total = []
    y_test_total = []
    
    # split the data into test and train
    kf = KFold(n_splits=10, shuffle=True, random_state=200)
    
    # kf fold cross val loop
    for train_index, test_index in kf.split(X):
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # scale train and tests
        scaler = StandardScaler().fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # fit KRR
        KRR = KernelRidge(kernel='rbf', alpha=alpha_value, gamma=gamma_value)
        
        y_pred = KRR.fit(X_train_scaled, y_train).predict(X_test_scaled)
        
        y_pred_total.append(y_pred)
        y_test_total.append(y_test)
        
    # flatten
    y_pred_total = [item for sublist in y_pred_total for item in sublist]
    y_test_total = [item for sublist in y_test_total for item in sublist]
    
    # calculate error metric
    rmse = np.sqrt(mean_squared_error(y_test_total, y_pred_total))
    #r_pearson = pearsonr(y_test_total, y_pred_total)
    #print('KRR k-fold cross-validation . alpha: %7.6f, gamma: %7.4f, RMSE: %7.4f, r: %7.4f' %(alpha_value,gamma_value,rmse,r_pearson))
    return rmse

In [67]:
# creating hyperparameters
def create_hyperparams_grid(X, y):
    
    graph_x  = []
    graph_y  = []    
    graph_z  = []    
    
    for alpha_value in np.arange(-5.0, 2.0, 0.7):
        
        alpha_value = pow(10, alpha_value)
        
        graph_x_row  = []
        graph_y_row  = []    
        graph_z_row  = []  
        
        for gamma_value in np.arange(0.0, 20, 2):
            
            hyperparams = (alpha_value, gamma_value)
            rmse = KRR_function(hyperparams, X, y)
            graph_x_row.append(alpha_value)
            graph_y_row.append(gamma_value)
            graph_z_row.append(rmse)
            
        graph_x.append(graph_x_row)
        graph_y.append(graph_y_row)
        graph_z.append(graph_z_row)
        print('')
        
    graph_x=np.array(graph_x)
    graph_y=np.array(graph_y)
    graph_z=np.array(graph_z)
    min_z = np.min(graph_z)
    
    pos_min_z = np.argwhere(graph_z == min_z)[0]
    
    print('Minimum RMSE: %.4f' %(min_z))
    print('Optimum alpha: %f' %(graph_x[pos_min_z[0],pos_min_z[1]]))
    print('Optimum gamma: %f' %(graph_y[pos_min_z[0],pos_min_z[1]]))

In [68]:
# Create {x1,x2,f} dataset every 1.0 from -10 to 10, with a noise of +/- 2
x1,x2,f=generate_data(-10,10,1.0,0.5)
# Prepare X and y for KRR
X,y = prepare_data_to_KRR(x1,x2,f)
# Create hyperparams grid
create_hyperparams_grid(X,y)











Minimum RMSE: 0.3425
Optimum alpha: 0.006310
Optimum gamma: 2.000000
